In [6]:
import os
import warnings
warnings.filterwarnings('ignore')
import torch
import torch.onnx
import bts.model as model
import bts.classifier as classifier

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
FILTER_LIST = [16,32,64,128,256]
TRAINED_MODEL_NAME = 'brain_tumor_segmentation_model_v2.pt'

unet_model = model.DynamicUNet(FILTER_LIST)
unet_classifier = classifier.BrainTumorClassifier(unet_model,device)
unet_classifier.restore_model(os.path.join('saved_models',TRAINED_MODEL_NAME))

# Create the 'saved_models' directory if it doesn't exist
os.makedirs('saved_models', exist_ok=True)

unet_model.eval()
dummy_input = torch.zeros(1, 1, 512, 512).to('cuda')
#     1 for the batch size (you can set this to 1 if you are dealing with single images at a time).
#     1 for the number of channels (grayscale images have one channel).
#     512 for the height of the image.
#     512 for the width of the image.
TRAINED_MODEL_NAME_ONNX = 'brain_tumor_segmentation_model_v3.onnx'
onnx_path = os.path.join('saved_models',TRAINED_MODEL_NAME_ONNX)

# Export the model to ONNX format
torch.onnx.export(unet_model, dummy_input, onnx_path, verbose=True)